In [2]:
import pandas as pd
import numpy as np

import os
wd = os.getcwd()
wd

'C:\\Users\\Nils\\Documents\\KIT-TVWL\\Master\\SS 18\\KD Seminar\\Git\\mlp'

In [107]:
df = pd.read_csv("../complete_data.csv")
df.Timestamp = pd.to_datetime(df.Timestamp)
df = df.drop(["Unnamed: 0", "ID"], axis=1)
df.head()

,Timestamp,articleCount,avgSentiment,stdSentiment,25quantileSentiment,50quantileSentiment,75quantileSentiment,maxSentiment,minSentiment,ibmSentiment,005930.KS,VIV,ALV.DE,TSS,BAYN.DE,TSLA,AIR.DE,AAPL,BAS.DE,BMW.DE
0,2018-05-02,896,0.081347,0.204661,-0.020812,0.068840,0.211710,0.94770,-0.562680,0,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
1,2018-05-02,156,0.190012,0.236579,0.119930,0.236800,0.323370,0.65250,-0.632340,0,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
2,2018-05-02,3116,0.050277,0.217317,-0.056716,0.029800,0.205750,0.92600,-0.886267,0,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
3,2018-05-02,510,0.068932,0.173565,-0.070315,0.072627,0.202486,0.64572,-0.503000,0,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
4,2018-05-02,250,0.055327,0.230707,-0.014180,0.030780,0.195380,0.60256,-0.574930,0,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682


In [112]:
df.shape
nr_days = df.shape[0] / 10
nr_feats = df.shape[1] -1 #(-1 wegen Timestamp)

In [113]:
df["Timestamp"].value_counts()

2018-05-04    10
2018-05-09    10
2018-05-02    10
2018-05-14    10
2018-05-10    10
2018-05-03    10
2018-05-15    10
2018-05-08    10
2018-05-11    10
Name: Timestamp, dtype: int64

In [114]:
X = []#np.empty((int(nr_days) ,int(nr_feats * 10)))

In [115]:
for d in df["Timestamp"].value_counts().index:
    X.append(df[df["Timestamp"] == d].drop(["Timestamp"], axis=1).values.reshape(1,nr_feats*10))
    

In [116]:
X = np.array(X).reshape(int(nr_days), nr_feats*10)
X.shape

(9, 190)

In [117]:
df_labels = df[["Timestamp","005930.KS","VIV","ALV.DE","TSS","BAYN.DE","TSLA","AIR.DE","AAPL","BAS.DE","BMW.DE"]].copy()
df_labels.head()

,Timestamp,005930.KS,VIV,ALV.DE,TSS,BAYN.DE,TSLA,AIR.DE,AAPL,BAS.DE,BMW.DE
0,2018-05-02,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
1,2018-05-02,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
2,2018-05-02,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
3,2018-05-02,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682
4,2018-05-02,0.0,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.01106,0.006682


In [118]:
df_labels_single = df_labels.groupby("Timestamp").agg("last")

In [119]:
df_labels_single.head()

,005930.KS,VIV,ALV.DE,TSS,BAYN.DE,TSLA,AIR.DE,AAPL,BAS.DE,BMW.DE
Timestamp,,,,,,,,,,
2018-05-02,0.000000,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.011060,0.006682
2018-05-03,0.000000,-0.020217,-0.009181,0.008611,0.003507,-0.057051,0.000612,0.001811,-0.017219,-0.010041
2018-05-04,-0.020973,-0.008056,0.008478,0.014659,0.018926,0.033328,0.006401,0.038483,0.012625,-0.006532
2018-05-08,0.013397,-0.010487,-0.003489,0.010593,-0.003532,-0.002646,-0.002593,0.004795,0.002798,0.004142
2018-05-09,-0.032853,-0.017471,-0.008826,0.005824,0.004120,0.016031,-0.010843,0.007016,0.002906,0.000218


In [120]:
Y = df_labels_single.values
Y.shape

(9, 10)

In [124]:
from keras.models import Sequential
from keras.layers import Dense, Activation

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [125]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)

In [126]:
dims = X.shape[1]
print(dims, 'dims')
print("Building model...")

nb_classes = Y.shape[1]
print(nb_classes, 'classes')

model = Sequential()
model.add(Dense(nb_classes, input_shape=(dims,), activation='sigmoid'))
model.add(Activation('softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.fit(X, Y, validation_data = (X_val, Y_val), epochs=20, 
          batch_size=128, verbose=True)

190 dims
Building model...
10 classes
Train on 9 samples, validate on 2 samples
Epoch 1/20
9/9 [==============================] - 0s 39ms/step - loss: -0.0385 - val_loss: -0.2599
Epoch 2/20
9/9 [==============================] - 0s 668us/step - loss: -0.0385 - val_loss: -0.2600
Epoch 3/20
9/9 [==============================] - 0s 445us/step - loss: -0.0385 - val_loss: -0.2621
Epoch 4/20
9/9 [==============================] - 0s 1ms/step - loss: -0.0390 - val_loss: -0.2952
Epoch 5/20
9/9 [==============================] - 0s 445us/step - loss: -0.0452 - val_loss: -0.2952
Epoch 6/20
9/9 [==============================] - 0s 775us/step - loss: -0.0452 - val_loss: -0.2952
Epoch 7/20
9/9 [==============================] - 0s 1ms/step - loss: -0.0454 - val_loss: -0.2952
Epoch 8/20
9/9 [==============================] - 0s 1ms/step - loss: -0.0470 - val_loss: -0.2952
Epoch 9/20
9/9 [==============================] - 0s 445us/step - loss: -0.0470 - val_loss: -0.2952
Epoch 10/20
9/9 [==========

In [135]:
X[0]

array([  5.51000000e+02,   8.21914827e-02,   2.06793140e-01,
        -1.01450000e-02,   4.14600000e-02,   2.13785000e-01,
         7.24430000e-01,  -6.32340000e-01,   0.00000000e+00,
        -2.09731234e-02,  -8.05570083e-03,   8.47804836e-03,
         1.46592056e-02,   1.89255091e-02,   3.33283517e-02,
         6.40083466e-03,   3.84833470e-02,   1.26254066e-02,
        -6.53170186e-03,   3.51000000e+02,  -6.38005924e-05,
         2.54183595e-01,  -1.81920000e-01,  -3.94000000e-02,
         2.19880000e-01,   6.17980000e-01,  -6.99633333e-01,
         0.00000000e+00,  -2.09731234e-02,  -8.05570083e-03,
         8.47804836e-03,   1.46592056e-02,   1.89255091e-02,
         3.33283517e-02,   6.40083466e-03,   3.84833470e-02,
         1.26254066e-02,  -6.53170186e-03,   2.04900000e+03,
         7.03780595e-02,   2.13552659e-01,  -2.62300000e-02,
         3.96700000e-02,   2.10740000e-01,   7.28870000e-01,
        -9.91700000e-01,   0.00000000e+00,  -2.09731234e-02,
        -8.05570083e-03,

In [140]:
E = model.predict(X[0].reshape(1,190))
E

array([[ 0.04923758,  0.04923761,  0.1338416 ,  0.1338416 ,  0.1338416 ,
         0.04923758,  0.1338416 ,  0.04923758,  0.1338416 ,  0.1338416 ]], dtype=float32)

In [137]:
Y[0]

array([ 0.        , -0.01066487,  0.01354925, -0.01383775,  0.00341881,
        0.00409271,  0.00747303,  0.04322714,  0.01106002,  0.0066825 ])

In [139]:
EY[0]

array([[ 0.04923758,  0.05990248,  0.12029236,  0.14767935,  0.1304228 ,
         0.04514487,  0.12636857,  0.00601043,  0.12278158,  0.1271591 ]])